In [1]:
 '''
Kathleen Kish
July 29, 2022
Script to build a mammalian retinal ganglion cell model with dendrites
Membrane dynamics from Fohlmeister et al. 2010 (https://journals.physiology.org/doi/full/10.1152/jn.00123.2009)

Code also allows for
    1. Intracellular current injection
    2. Extracellular stimulation from an ideal point source electrode
    3. Extracellular stimulation from a disc electrode (where extracellular voltage is imported from a .txt file)

'''

import numpy as np
import matplotlib.pyplot as plt
from neuron import h,gui
from mpl_toolkits.mplot3d import Axes3D

h.load_file('interpxyz.hoc')
h.load_file('setpointers.hoc')

After any change to cell geometry or nseg, be sure to invoke setpointers()


1.0

In [2]:
######################################################
# BUILD RGC
######################################################

# Import dendritic tree and soma from SWC file
# Source: Neuromorpho database - reference https://pubmed.ncbi.nlm.nih.gov/30524239/
# Human retinal ganglion cell
# Cell ID: 151109_B13_2.swc

class Local_Cell(object):
    def __init__(self, cell_file): # ---- >> OTHER PARAMETERS, k_matrix_file, rot_theta, rot_phi):
        h.load_file('stdrun.hoc')
        self.load_cell(cell_file)
            
    def load_cell(self,file_path):
        h.load_file(file_path)
        self.cell = h.Cell() ## Create cell from the template created in the ".hoc" file
        self.all = h.SectionList() ## Create section list

RGC = Local_Cell('RGC_morph_4.hoc')

# Create an Elliptical Axon Trajectory for the RGC 

%run Axon_Ellipse.ipynb

StartingCoordinates = (-.01194,0,.020) # mm <---- top compartment of the soma
NFLEntryCoordinates = (-0.31194,0,.040) # mm <---- 300 um left and 20 um above
Length = 3051 # um

axon_coords = Build_Single_RGC(StartingCoordinates,NFLEntryCoordinates,Length) # calculates ellipse trajectory in mm
xpts_axon = axon_coords[:,0]*(1000) # um
ypts_axon = axon_coords[:,1]*(1000) # um
zpts_axon = axon_coords[:,2]*(1000) # um

# Add Membrane Dynamics to the RGC

h.celsius=37.1 # temperature (deg Celsius)

####### dendrites 
# morphology/connections already exist
# biophysics
for i in np.arange(119):
    RGC.cell.dend[i].nseg = int(np.round((RGC.cell.dend[i].L)/10)+1)
    RGC.cell.dend[i].cm = 1 # membrane capacitance
    RGC.cell.dend[i].Ra = 136.6 # axial resistance (ohm-cm)
    RGC.cell.dend[i].insert('pas') 
    RGC.cell.dend[i].e_pas = -65.02 # passive membrane potential (mV)
    RGC.cell.dend[i].g_pas = .0001 # leakage conductance (S/cm^2)
    RGC.cell.dend[i].insert('mammalian_spike') 
    RGC.cell.dend[i].ena = 61.02 # sodium resting potential (mV)
    RGC.cell.dend[i].ek = -102.03 # potassium resting potential (mV)
    RGC.cell.dend[i].gnabar_mammalian_spike = 0.060 # maximum sodium conductance (S/cm^2)
    RGC.cell.dend[i].gkbar_mammalian_spike = 0.035 # maximum potassium delayed rectifier conductance (S/cm^2)
    RGC.cell.dend[i].gcabar_mammalian_spike = 0.001 # maximum calcium conductance (S/cm^2)
    RGC.cell.dend[i].gkcbar_mammalian_spike = 0.00017   # maximum calcium-dependent potassium conductance (S/cm^2)  
    RGC.cell.dend[i].insert('cad') 
    RGC.cell.dend[i].depth_cad = 0.1 # calcium pump depth (microns)
    RGC.cell.dend[i].taur_cad = 1.5 # time constant (msec)
    RGC.cell.dend[i].insert('extracellular') 
    RGC.cell.dend[i].xg[0] = 1e10 # membrane resistance (S/cm^2)
    RGC.cell.dend[i].e_extracellular = 0 # extracellular voltage (mV)
    RGC.cell.dend[i].insert('xtra')

####### soma 
# morphology/connections already exist
# biophysics
RGC.cell.soma.nseg = 5
RGC.cell.soma.cm = 1 # membrane capacitance
RGC.cell.soma.diam = 20 # section diameter (microns)
RGC.cell.soma.Ra = 136.6 # soma axial resistance (ohm-cm)
RGC.cell.soma.insert('pas') 
RGC.cell.soma.e_pas = -65.02 # passive membrane potential (mV)
RGC.cell.soma.g_pas = .0001 # leakage conductance (S/cm^2)
RGC.cell.soma.insert('mammalian_spike') 
RGC.cell.soma.ena = 61.02 # sodium resting potential (mV)
RGC.cell.soma.ek = -102.03 # potassium resting potential (mV)
RGC.cell.soma.gnabar_mammalian_spike = 0.060  # maximum sodium conductance (S/cm^2)
RGC.cell.soma.gkbar_mammalian_spike = 0.035 # maximum potassium delayed rectifier conductance (S/cm^2)
RGC.cell.soma.gcabar_mammalian_spike = 0.00075 # maximum calcium conductance (S/cm^2)
RGC.cell.soma.gkcbar_mammalian_spike = 0.00017  # maximum calcium-dependent potassium conductance (S/cm^2) 
RGC.cell.soma.insert('cad') 
RGC.cell.soma.depth_cad = 0.1 # calcium pump depth (microns)
RGC.cell.soma.taur_cad = 1.5 # time constant (msec)
RGC.cell.soma.insert('extracellular') 
RGC.cell.soma.xg[0] = 1e10 # membrane resistance (S/cm^2)
RGC.cell.soma.e_extracellular = 0 # extracellular voltage
RGC.cell.soma.insert('xtra')
    
####### axon hillock (AH)
h('create AH')
# biophysics
h.AH.nseg = 8
h.AH.cm = 1 # membrane capacitance
h.AH.Ra = 136.6 # axial resistance (ohm-cm)
h.AH.insert('pas') 
h.AH.e_pas = -65.02 # passive membrane potential (mV)
h.AH.g_pas = .0001 # leakage conductance (S/cm^2)
h.AH.insert('mammalian_spike') 
h.AH.ena = 61.02 # sodium resting potential (mV)
h.AH.ek = -102.03 # potassium resting potential (mV)
h.AH.gnabar_mammalian_spike = 0.150 # maximum sodium conductance (S/cm^2)
h.AH.gkbar_mammalian_spike = 0.090 # maximum potassium delayed rectifier conductance (S/cm^2)
h.AH.gcabar_mammalian_spike = 0.00075 # maximum calcium conductance (S/cm^2)
h.AH.gkcbar_mammalian_spike = 0.00017  # maximum calcium-dependent potassium conductance (S/cm^2) 
h.AH.insert('cad')
h.AH.depth_cad = 0.1 # calcium pump depth (microns)
h.AH.taur_cad = 1.5 # time constant (msec)
h.AH.insert('extracellular') 
h.AH.xg[0] = 1e10 # membrane resistance (S/cm^2)
h.AH.e_extracellular = 0 # extracellular voltage (mV)
h.AH.insert('xtra')
# connections
h.AH.connect(RGC.cell.soma) # connect the first section of the axon hillock back to the soma
# morphology
h.AH.pt3dclear()
for i in np.arange(41):
    AH_diameter = 3
    h.AH.pt3dadd(xpts_axon[i],ypts_axon[i],zpts_axon[i],AH_diameter)

####### sodium channel band (SOCB)
h('create SOCB')
# biophysics
h.SOCB.nseg = 8
h.SOCB.cm = 1 # membrane capacitance (uF/cm2)
h.SOCB.Ra = 136.6 # axial resistance (ohm-cm)
h.SOCB.insert('pas') 
h.SOCB.e_pas = -65.02 # passive membrane potential (mV)
h.SOCB.g_pas = .0001 # leakage conductance (S/cm^2)
h.SOCB.insert('mammalian_spike') 
h.SOCB.ena = 61.02 # sodium resting potential (mV)
h.SOCB.ek = -102.03 # potassium resting potential (mV)
h.SOCB.gnabar_mammalian_spike = 0.420 # maximum sodium conductance (S/cm^2)
h.SOCB.gkbar_mammalian_spike =  0.250 # maximum potassium delayed rectifier conductance (S/cm^2)
h.SOCB.gcabar_mammalian_spike = 0.00075 # maximum calcium conductance (S/cm^2)
h.SOCB.gkcbar_mammalian_spike = 0.00011 # maximum calcium-dependent potassium conductance (S/cm^2) 
h.SOCB.insert('cad')
h.SOCB.depth_cad = 0.1 # calcium pump depth (microns)
h.SOCB.taur_cad = 1.5 # time constant (msec)
h.SOCB.insert('extracellular') 
h.SOCB.xg[0] = 1e10 # membrane resistance (S/cm^2)
h.SOCB.e_extracellular = 0 # extracellular voltage
h.SOCB.insert('xtra')
# connections
h.SOCB.connect(h.AH) # connect the first section of the SOCB back to the axon hillock
# morphology 
h.SOCB.pt3dclear()
for i in np.arange(41):
    SOCB_diameter = 3-(0.055*i) 
    h.SOCB.pt3dadd(xpts_axon[i+40],ypts_axon[i+40],zpts_axon[i+40],SOCB_diameter)

####### narrow region (NR)
h('create NR')
# biophysics
h.NR.nseg = 15
h.NR.cm = 1 # membrane capacitance (uF/cm2)
h.NR.Ra = 136.6 # axon hillock axial resistance (ohm-cm)
h.NR.insert('pas') 
h.NR.e_pas = -65.02 # passive membrane potential (mV)
h.NR.g_pas = .0001 # leakage conductance (S/cm^2)
h.NR.insert('mammalian_spike') 
h.NR.ena = 61.02 # sodium resting potential (mV)
h.NR.ek = -102.03 # potassium resting potential (mV)
h.NR.gnabar_mammalian_spike = 0.150 # maximum sodium conductance (S/cm^2)
h.NR.gkbar_mammalian_spike = 0.090 # maximum potassium delayed rectifier conductance (S/cm^2)
h.NR.gcabar_mammalian_spike = 0.00075 # maximum calcium conductance (S/cm^2)
h.NR.gkcbar_mammalian_spike = 0.0002  # maximum calcium-dependent potassium conductance (S/cm^2) 
h.NR.insert('cad')
h.NR.depth_cad = 0.1 # calcium pump depth (microns)
h.NR.taur_cad = 1.5 # time constant (msec)
h.NR.insert('extracellular') 
h.NR.xg[0] = 1e10 # membrane resistance (S/cm^2)
h.NR.e_extracellular = 0 # extracellular voltage
h.NR.insert('xtra')
# connections
h.NR.connect(h.SOCB) # connect the first section of the narrow region back to the SOCB
# morphology 
h.NR.pt3dclear()
for i in np.arange(76):
    NR_diameter = 0.8
    h.NR.pt3dadd(xpts_axon[i+80],ypts_axon[i+80],zpts_axon[i+80],NR_diameter)

####### distal axon
h('create axon')
# biophysics
h.axon.nseg = 579
h.axon.diam = 1 # section diameter (microns)
h.axon.cm = 1 # membrane capacitance (uF/cm2)
h.axon.Ra = 136.6 # axial resistance (ohm-cm)
h.axon.insert('pas') 
h.axon.e_pas = -65.02 # passive membrane potential (mV)
h.axon.g_pas = .0001 # leakage conductance (S/cm^2)
h.axon.insert('mammalian_spike') 
h.axon.ena = 61.02 # sodium resting potential (mV)
h.axon.ek = -102.03 # potassium resting potential (mV)
h.axon.gnabar_mammalian_spike = 0.100 # maximum sodium conductance (S/cm^2)
h.axon.gkbar_mammalian_spike = 0.050 # maximum potassium delayed rectifier conductance (S/cm^2)
h.axon.gcabar_mammalian_spike = 0.00075 # maximum calcium conductance (S/cm^2)
h.axon.gkcbar_mammalian_spike = 0.0002   # maximum calcium-dependent potassium conductance (S/cm^2) 
h.axon.insert('cad') 
h.axon.depth_cad = 0.1 # calcium pump depth (microns)
h.axon.taur_cad = 1.5 # time constant (msec)
h.axon.insert('extracellular') 
h.axon.xg[0] = 1e10 # membrane resistance (S/cm^2)
h.axon.e_extracellular = 0 # extracellular voltage
h.axon.insert('xtra')
# connections
h.axon.connect(h.NR) # connect the first section of the distal axon back to the narrow region
# morphology
h.axon.pt3dclear()
for i in np.arange(2896):
    h.axon.pt3dadd(xpts_axon[i+155],ypts_axon[i+155],zpts_axon[i+155],1)

h.setpointers()

0.0

In [3]:
######################################################
# CURRENT CLAMP - inject intracellular current to verify cell model
######################################################

h.init()
h.v_init = -70 # mV
h.finitialize(h.v_init)
h.fcurrent()
h.dt = 0.005 # msec 
h.tstop = 50 # msec
    
# Record time
t_vec=h.Vector().record(h._ref_t) 

# Record membrane voltage at various compartments
v_vec_dend=h.Vector().record(RGC.cell.dend[50](0.5)._ref_v) 
v_vec_soma=h.Vector().record(RGC.cell.soma(0.5)._ref_v) 
v_vec_AH=h.Vector().record(h.AH(0.5)._ref_v) 
v_vec_SOCB=h.Vector().record(h.SOCB(0.5)._ref_v) 
v_vec_NR=h.Vector().record(h.NR(0.5)._ref_v)
v_vec_axon=h.Vector().record(h.axon(0.5)._ref_v)

# Create intracellular stimulus
CurrentClamp = h.IClamp(0.5,sec=RGC.cell.soma) # inject current to soma
CurrentClamp.delay= 10 # msec
CurrentClamp.dur= 1 # msec
CurrentClamp.amp= 1 # nA

h.run()

# Membrane Voltage Plot
%matplotlib qt
fig=plt.figure()
plt.plot(t_vec,v_vec_soma,label='Soma')
plt.plot(t_vec,v_vec_SOCB,label='SOCB')
plt.plot(t_vec,v_vec_axon,label='Axon')
plt.xlabel('Time (msec)')
plt.ylabel('Membrane Voltage (mV)')
plt.legend()

In [5]:
# ######################################################
# # POINT SOURCE STIMULATION
# ######################################################

%run Extracellular_Stimulation.ipynb

electrode = (-100, 0, 105) # x,y,z position of the point source (um)
sigma = 0.1 # tissue conductivity (S/m)

# Define stimulus waveform
PW = 0.45 # msec
frequency = 20 # Hz
tstop = 50 # msec
dt = 0.005 # msec 
pulse_shape = stimulus(PW,frequency,tstop,dt)

x_coords=[]
y_coords=[]
z_coords=[]

for sec in h.allsec():
    for seg in sec:
        x_coords.append(seg.x_xtra) # (um)
        y_coords.append(seg.y_xtra) # (um)
        z_coords.append(seg.z_xtra) # (um)

voltages = [] 
for i in np.arange(len(x_coords)):
    x_dist = (x_coords[i]-electrode[0])*(1e-6) # convert to (m)
    y_dist = (y_coords[i]-electrode[1])*(1e-6)
    z_dist = (z_coords[i]-electrode[2])*(1e-6)
    distance = np.sqrt((x_dist**2)+(y_dist**2)+(z_dist**2))
    voltage = 1 / (4 * np.pi * sigma * distance) # ideal point source equation
    voltages.append(voltage)

# # Cell Shape Plot
# %matplotlib qt
# fig=plt.figure()
# for i in np.arange(len(x_coords)):
#     plt.plot(x_coords,z_coords,'ok')
#     plt.plot([electrode[0]], [electrode[2]], 'or')
# plt.xlabel('X-Coordinate (um)')
# plt.ylabel('Z-Coordinate (um)')

# #  Extracellular Voltage Plot
# %matplotlib qt
# fig=plt.figure()
# plt.plot(voltages,'o')
# plt.xlabel('Compartment')
# plt.ylabel('Extracellular Voltage (V)')

# Record time
t_vec=h.Vector().record(h._ref_t) 

# Record membrane voltage at various compartments
v_vec_soma=h.Vector().record(RGC.cell.soma(0.5)._ref_v) 
v_vec_SOCB=h.Vector().record(h.SOCB(0.5)._ref_v) 
v_vec_axon=h.Vector().record(h.axon(0.5)._ref_v)

# # Run simulation
scaling_var = 4 # Stimulation current amplitude (uA)
integrate(tstop, dt, pulse_shape, voltages, scaling_var*(1e-3))

# Membrane Voltage Plot
%matplotlib qt
fig=plt.figure()
plt.plot(t_vec,v_vec_soma,label='Soma')
plt.plot(t_vec,v_vec_SOCB,label='SOCB')
plt.plot(t_vec,v_vec_axon,label='Axon')
plt.xlabel('Time (msec)')
plt.ylabel('Membrane Voltage (mV)')
plt.legend()

In [21]:
######################################################
# DISC ELECTRODE STIMULATION - import voltage .txt file from COMSOL
######################################################

%run Extracellular_Stimulation.ipynb

voltage_file = 'Voltages_from_COMSOL.txt'
voltage_export = load_txt_file(voltage_file,'object')

cell_number = 1
cell_length = 959 

data = [] # Array to hold all the information from COMSOL
for i in np.arange(len(voltage_export)):
    first_character = voltage_export[i][0]
    if first_character != '%': # Remove comments at the beginning of exported file
        data.append(voltage_export[i]) 
voltages = [] # Array to hold voltages
for j in np.arange(len(data)):
    voltages.append(data[j][3])

voltages = np.array(np.reshape(voltages,(cell_number,cell_length)),dtype=np.float)  # Re-shape the voltage array to make it 3-d; each level is a cell

# #  Extracellular Voltage Plot
# %matplotlib qt
# fig=plt.figure()
# plt.plot(voltages[0],'o')
# plt.xlabel('Compartment')
# plt.ylabel('Extracellular Voltage (V)')

# Record time
t_vec=h.Vector().record(h._ref_t) 

# Record membrane voltage at various compartments
v_vec_soma=h.Vector().record(RGC.cell.soma(0.5)._ref_v) 
v_vec_SOCB=h.Vector().record(h.SOCB(0.5)._ref_v) 
v_vec_axon=h.Vector().record(h.axon(0.5)._ref_v)

# # Run simulation
scaling_var = 37 # Stimulation current amplitude (uA)
integrate(tstop, dt, pulse_shape, voltages[0], scaling_var*(1e-3))

# Membrane Voltage Plot
%matplotlib qt
fig=plt.figure()
plt.plot(t_vec,v_vec_soma,label='Soma')
plt.plot(t_vec,v_vec_SOCB,label='SOCB')
plt.plot(t_vec,v_vec_axon,label='Axon')
plt.xlabel('Time (msec)')
plt.ylabel('Membrane Voltage (mV)')
plt.legend()